In [6]:
import pandas as pd

def get_excel_sheet_names(file_path):
    try:
        # Read the Excel file
        excel_file = pd.ExcelFile(file_path)
        
        # Get all sheet names
        sheet_names = excel_file.sheet_names
        
        return sheet_names
    
    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Example usage
file_path = 'IVM6201_ATE_TM.xlsx'
sheets = get_excel_sheet_names(file_path)

print("Sheets in the Excel file:")
for sheet in sheets:
    print(sheet)

Sheets in the Excel file:
Keywords
Procedure
Trimming
Reference
LDO5
PLL
Short_Diag
ADDSEL_HWMUTE
CP
Buck
DC Diagnostic
AC Diagnostic
FLL
PGA-I Ch1
PGA-I Ch2
PGA-I Ch3
PGA-I Ch4
PGA-V Ch1
PGA-V Ch2
PGA-V Ch3
PGA-V Ch4
Class-D Signal Ch1
Drivers CLD Ch1
Class-D Signal Ch2
Drivers CLD Ch2
Class-D Signal Ch3
Drivers CLD Ch3
Class-D Signal Ch4
Drivers CLD Ch4
OTP
VISENSE_COEFF
Foglio2


In [7]:
import pandas as pd 
from dft import parse_multiplier_value
procedures_df = pd.read_excel('IVM6201_ATE_TM.xlsx', sheet_name='Procedure')
df = pd.read_excel('IVM6201_ATE_TM.xlsx', sheet_name='CP')
raw_data = df.iloc[:, :] # make shallow copy of the raw data
# raw_data.iloc[0,1] = raw_data.iloc[0,0] # copy from first column to second column
# raw_data.iloc[1,1] = raw_data.iloc[1,0] # copy from first column to second column
# raw_data.iloc[2,1] = raw_data.iloc[2,0] # copy from first column to second column
# raw_data.iloc[3,1] = raw_data.iloc[3,0] # copy from first column to second column
# raw_data.iloc[4,1] = raw_data.iloc[4,0] # copy from first column to second column
# raw_data.iloc[5,1] = raw_data.iloc[5,0] # copy from first column to second column
# raw_data.drop(columns=df.columns[0], axis=1, inplace=True)
raw_data.columns = [ x.strip().replace(' ', '_') if isinstance(x, str) else x for x in raw_data.iloc[3].tolist()]
raw_data.set_index(raw_data.columns[0], inplace=True)
raw_data.loc['Typ'] = raw_data.loc['Typ'].apply(lambda x : parse_multiplier_value(x))
raw_data.loc['Min'] = raw_data.loc['Min'].apply(lambda x : parse_multiplier_value(x))
raw_data.loc['Max'] = raw_data.loc['Max'].apply(lambda x : parse_multiplier_value(x))
# raw_data.head(20)

c:\Users\harkum\Documents\6201\env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\harkum\Documents\6201\env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
print(raw_data.columns.to_list())

['NL_ron', 'PL_ron', 'NH_ron', 'PH_ron', 'CP_PGOOD', 'Startup_Current', 'IABSP2N', 'IABSN2P', 'IDISCHARGE', 'Vout_Functional']


In [9]:
from dft import *
from common import ivm6201_pin_check

for test_name in (procedures:=raw_data.columns.to_list()):
    print('*'*10, test_name, '*'*10)
    for instruction in raw_data.loc['Instructions' ,test_name].split('\n'):
        if (procedure := parse_procedure_name(instruction)):
            if procedure in procedures_df.columns.to_list():
                # print('Procedure name :>',  procedure)
                pass
            else:
                print(f'!!!!!Procedure Failed {procedure}!!!!!!!!!')
        elif (delay := parse_wait_delay(instruction)):
            # print('Delay operation :>', delay)
            pass
        elif (const_value := parse_constant_value(instruction)):
            # print('Const Value  :>', const_value)
            pass
        elif (register := parse_register_notation(instruction)):
            # print('Register operation :>', register)
            pass
        elif (force_sweep := parse_force_sweep_instruction(instruction)):
            # print('force_sweep operation :>', force_sweep)
            if (primay_signal := force_sweep.get('primary_signal')) and (ivm6201_pin_check(primay_signal)):
                if (secondary_signal := force_sweep.get('secondary_signal')): 
                    if ivm6201_pin_check(secondary_signal):
                        pass
                    else:
                        print(f'!!!!! force_sweep secondary fail Signal pin Dose not Exists: {secondary_signal} , {force_sweep}')
                    
            else:
                print(f'!!!!! force_sweep primary fail Signal pin Dose not Exists: {primay_signal} , {force_sweep}')
            pass
        elif (force := parse_force_instruction(instruction)):
            # print('force operation :>', force)
            if (primay_signal := force.get('primary_signal')) and (ivm6201_pin_check(primay_signal)):
                if (secondary_signal := measrement.get('secondary_signal')): 
                    if ivm6201_pin_check(secondary_signal):
                        pass
                    else:
                        print(f'!!!!! force secondary fail Signal pin Dose not Exists: {secondary_signal} , {measrement}')
                    
            else:
                print(f'!!!!! force primary fail Signal pin Dose not Exists: {primay_signal} , {measrement}')
            pass
        elif (savemeas := parse_savemeas(instruction)):
            # print('savemeas operation :>', savemeas)
            if (primay_signal := savemeas.get('primary_signal')) and (ivm6201_pin_check(primay_signal)):
                if (secondary_signal := savemeas.get('secondary_signal')): 
                    if ivm6201_pin_check(secondary_signal):
                        pass
                    else:
                        print(f'!!!!! savemeas secondary fail Signal pin Dose not Exists: {secondary_signal} , {measrement}')
                    
            else:
                print(f'!!!!! savemeas primary fail Signal pin Dose not Exists: {primay_signal} , {measrement}')
            pass
            pass
        elif (measrement := parse_measurements(instruction)):
            # print('Measure operation :>', measrement)
            if (primay_signal := measrement.get('primary_signal')) and (ivm6201_pin_check(primay_signal)):
                if (secondary_signal := measrement.get('secondary_signal')): 
                    if ivm6201_pin_check(secondary_signal):
                        pass
                    else:
                        print(f'!!!!! measurement secondary fail Signal pin Dose not Exists: {secondary_signal} , {measrement}')
                    
            else:
                print(f'!!!!! measurement primary fail Signal pin Dose not Exists: {primay_signal} , {measrement}')
            pass

        elif (trigger := parse_trigger_instruction(instruction)):
            # print('trigger operation :>', trigger)
            pass
        elif (trim_reg := parse_trim_instruction(instruction)):
            # print('Trim_reg operation :>', trim_reg)
            pass
        elif (meas_match := parse_meas_match_regex(instruction)):
            # print('meas_match operation :>', meas_match)
            pass
        elif (calculate := parse_calculate_expression(instruction)):
            # print('calculate operation :>', calculate)
            pass
        elif (sweep_trig_store := parse_sweep_trig_store(instruction)):
            # print(sweep_trig_store)
            # {'sweep_signal': 'AVDD', 'sweeper_reference': 'GND', 'initial_value': 5.0, 'final_value': 3.0, 'step_size': 0.1, 'sweep_time': 0.01, 'unit': 'V', 'trig_signal': 'SDO', 'trig_reference': 'GND', 'trig_state': 'HL', 'trig_value': 0, 'variable': 'Low_Supply_mute'}
            if (sweep_signal := ivm6201_pin_check(sweep_trig_store.get('sweep_signal'))) and \
                (sweep_reference:=ivm6201_pin_check(sweep_trig_store.get('sweeper_reference'))) and \
                (trig_signal := ivm6201_pin_check(sweep_trig_store.get('trig_signal'))) and \
                (trig_reference := ivm6201_pin_check(sweep_trig_store.get('trig_reference'))) :
                    pass
            else:
                
                print('sweep_trig_store pin check failed :>', sweep_trig_store.get('sweep_signal'), sweep_trig_store.get('sweeper_reference'),  sweep_trig_store.get('trig_signal'), sweep_trig_store.get('trig_reference'))
            pass
    
        else:
            if not re.match(r'"(?:[^\\"]|\\.)*"', instruction) and instruction:
                
                print('fail :', instruction)
        
    print(f" min limit: {raw_data.loc['Min',test_name]}\n typ limit: {raw_data.loc['Typ',test_name]}\n max limit: {raw_data.loc['Max',test_name]}   ")
    print('@'*20)

********** NL_ron **********
 min limit: 0.3
 typ limit: 0.5
 max limit: 0.8   
@@@@@@@@@@@@@@@@@@@@
********** PL_ron **********
 min limit: 0.7
 typ limit: 1
 max limit: 1.5   
@@@@@@@@@@@@@@@@@@@@
********** NH_ron **********
 min limit: 0.3
 typ limit: 0.5
 max limit: 0.8   
@@@@@@@@@@@@@@@@@@@@
********** PH_ron **********
 min limit: 0.7
 typ limit: 1
 max limit: 1.5   
@@@@@@@@@@@@@@@@@@@@
********** CP_PGOOD **********
 min limit: nan
 typ limit: HL= 4.16
LH= 3.34
 max limit: nan   
@@@@@@@@@@@@@@@@@@@@
********** Startup_Current **********
 min limit: 0.015
 typ limit: 0.01
 max limit: 0.005   
@@@@@@@@@@@@@@@@@@@@
********** IABSP2N **********
 min limit: 6e-07
 typ limit: 1e-06
 max limit: 1.4e-06   
@@@@@@@@@@@@@@@@@@@@
********** IABSN2P **********
 min limit: 6e-07
 typ limit: 1e-06
 max limit: 1.4e-06   
@@@@@@@@@@@@@@@@@@@@
********** IDISCHARGE **********
 min limit: 0.02
 typ limit: 0.024
 max limit: 0.03   
@@@@@@@@@@@@@@@@@@@@
********** Vout_Functional **********
!

In [ ]:
# # from test_analyzer import TestAnalyzer
# import subprocess

# for test_name in ['NL_ron', 'PL_ron', 'NH_ron', 'PH_ron', 'CP_PGOOD', 'Startup_Current', 'IABSP2N', 'IABSN2P', 'IDISCHARGE', 'Vout_Functional']:
#     subprocess.run(["python", 'test_analyzer.py',"--excel_file","IVM6201_ATE_TM.xlsx", "--sheet_name","CP", "--test_name",test_name])
import test_analyzer
analyze = test_analyzer.TestAnalyzer(excel_file="IVM6201_ATE_TM.xlsx", sheet_name="CP", test_name='CP_PGOOD')
analyze.analyze_test()

c:\Users\harkum\Documents\6201\env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\harkum\Documents\6201\env\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
